In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import sklearn
import os
import PyPDF2
import pdfplumber
import fitz
import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

In [14]:
from concurrent.futures import ThreadPoolExecutor

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file using pdfplumber."""
    with pdfplumber.open(pdf_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

def preprocess_text(text):
    """Clean the extracted text for easier processing."""
    text = re.sub(r'\n+', '\n', text)  # Remplacer les sauts de ligne multiples par un seul
    text = re.sub(r'\s+', ' ', text)  # Remplacer les espaces multiples par un seul
    return text

def extract_information(text):
    """Attempt to extract title, author, date, and other info from the text."""
    # Rechercher les sections du texte où se trouvent les informations
    title = re.search(r"(Title|Titre):?\s*(.*)", text, re.IGNORECASE)
    if title:
        title = title.group(2)
    else:
        # Si la première méthode ne fonctionne pas, on essaie de prendre la première ligne
        title = text.split('\n')[0][:100]  # Prendre les premiers 100 caractères comme titre

    author = re.search(r"(Author|Auteur):?\s*(.*)", text, re.IGNORECASE)
    if author:
        author = author.group(2)
    else:
        author = "Auteur non trouvé"

    # Extraction de la date (en supposant un format standard comme YYYY-MM-DD, DD/MM/YYYY, etc.)
    date = re.search(r"(\b\d{4}[-/]\d{2}[-/]\d{2}\b|\b\d{2}[-/]\d{2}[-/]\d{4}\b)", text)
    if date:
        date = date.group(0)
    else:
        date = "Date non trouvée"

    # Extraction du contenu après les premières informations trouvées
    content_start = text.find(title) + len(title) if title else 0
    content = text[content_start:].strip()

    return {
        "title": title.strip(),
        "author": author.strip(),
        "date": date.strip(),
        "content": content
    }

def process_single_pdf(pdf_file, pdf_folder):
    pdf_path = os.path.join(pdf_folder, pdf_file)
    text = extract_text_from_pdf(pdf_path)
    cleaned_text = preprocess_text(text)
    info = extract_information(cleaned_text)
    info["file_name"] = pdf_file  # Ajouter le nom du fichier pour référence
    return info

def process_pdfs(pdf_folder):
    pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith(".pdf")]
    results = []

    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_single_pdf, pdf_file, pdf_folder) for pdf_file in pdf_files]
        for future in futures:
            results.append(future.result())

    # Créer un DataFrame à partir des résultats
    df = pd.DataFrame(results)
    return df

pdf_folder_path = "articles et bouquins"
pdf_data_df = process_pdfs(pdf_folder_path)


In [2]:
# Afficher les résultats
pdf_data_df = pd.read_csv("corpus.csv")
pdf_data_df.head()

,title,author,date,content,file_name
0,Une am´elioration de la convergence de la m´et...,Email addresses: fatimabouyghf3@gmail.com(F.BO...,Date non trouvée,"LabMIA-SI, University of Mohammed V Rabat, Mor...",Article_11.pdf
1,L’am´elioration de la convergence de la m´etho...,Email address: fatimabouyghf3@gmail.com(F.BOUY...,Date non trouvée,e d’un seul ou plusieurs second membres F. BOU...,Article_22.pdf
2,Une approche unifi´ee pour les m´ethodes de so...,Email addresses: fatimabouyghf3@gmail.com(F.BO...,Date non trouvée,"lin´eaires F. BOUYGHFa,b, A. MESSAOUDIa, H. SA...",Article_33.pdf
3,1 Appendix ThecoefficientmatrixAandtheright–ha...,Auteur non trouvé,Date non trouvée,onjugate GradientMethod” by Hao Ji and Yaohang...,BFBCGappendix.pdf
4,Electronic Transactions on Numerical Analysis....,Auteur non trouvé,Date non trouvée,"ity Copyright2009, Kent State University. htt...",bloc sadok messaoudi.pdf


In [16]:
# Exporter les données vers un fichier CSV si nécessaire
pdf_data_df.to_csv("corpus.csv", index=False)

In [3]:
# Télécharger les stop words de nltk si ce n'est pas déjà fait
nltk.download('stopwords')
nltk.download('punkt')

# Obtenir les stop words en français et en anglais
stop_words = set(stopwords.words('french') + stopwords.words('english'))

def clean_text(text):
    """Nettoyer et prétraiter le texte."""
    if not isinstance(text, str):
        return ""
    # 1. Suppression des balises HTML
    text = BeautifulSoup(text, "html.parser").get_text()
    
    # 2. Mettre en minuscules
    text = text.lower()
    
    # 3. Suppression de la ponctuation
    text = re.sub(f"[{re.escape(string.punctuation)}]", " ", text)
    
    # 4. Tokenisation
    tokens = word_tokenize(text)
    
    # 5. Suppression des stop words
    tokens = [word for word in tokens if word not in stop_words]
    
    # 6. Rejoindre les tokens nettoyés
    cleaned_text = " ".join(tokens)
    return cleaned_text

# Appliquer le nettoyage sur les colonnes 'title' et 'content'
pdf_data_df['cleaned_title'] = pdf_data_df['title'].apply(clean_text)
pdf_data_df['cleaned_content'] = pdf_data_df['content'].apply(clean_text)

# Afficher les premières lignes pour vérifier le nettoyage
pdf_data_df[['cleaned_title', 'cleaned_content']].head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Maxim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Maxim\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,cleaned_title,cleaned_content
0,am´elioration convergence m´ethode idr f bouyg...,labmia si university mohammed v rabat morocco ...
1,’ am´elioration convergence m´ethode bicgstab ...,e ’ seul plusieurs second membres f bouyghfa b...
2,approche unifi´ee m´ethodes sous espace krylov...,lin´eaires f bouyghfa b messaoudia h sadokb la...
3,1 appendix thecoefficientmatrixaandtheright–ha...,onjugate gradientmethod ” hao ji yaohangli 1 a...
4,electronic transactions numerical analysis etn...,ity copyright2009 kent state university http ...


In [13]:
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize

# Tokeniser le contenu de chaque document pour BM25
corpus = pdf_data_df['cleaned_content'].tolist()
tokenized_corpus = [word_tokenize(doc) for doc in corpus]

# Initialiser le modèle BM25
bm25 = BM25Okapi(tokenized_corpus)

# Fonction pour classer les documents en fonction d'une requête
def bm25_search(query, bm25, corpus_df, top_n=5):
    tokenized_query = word_tokenize(query.lower())
    scores = bm25.get_scores(tokenized_query)
    corpus_df['bm25_score'] = scores
    # Obtenir les top n résultats triés par score
    return corpus_df.sort_values(by='bm25_score', ascending=False).head(top_n)

query = "machine learning"
top_results_bm25 = bm25_search(query, bm25, pdf_data_df)
print(top_results_bm25[['title', 'bm25_score']])


                                                title  bm25_score
23  Numerische Mathematik Bd. 1, S. 29-- 37 (I 959...    2.516734
43  Numerical Algorithms20 (1999) 303–321 303 CMRH...    1.694497
24  NUMERICAL EXPERIMENTS WITH A MULTIPLE GRID AND...    1.674265
6   SIAMJ.MATH.ANAL. (cid:2)c 2010Societ yforIndus...    1.539991
14  See discussions, stats, and author profiles fo...    1.091323


In [16]:
from sentence_transformers import SentenceTransformer, util
import torch

# Charger un modèle BERT pré-entraîné
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Encoder tout le contenu des documents en embeddings
corpus_embeddings = model.encode(pdf_data_df['cleaned_content'].tolist(), convert_to_tensor=True)

# Fonction pour effectuer une recherche sémantique
def semantic_search(query, model, corpus_embeddings, corpus_df, top_n=5):
    query_embedding = model.encode(query, convert_to_tensor=True)
    # Calculer les similarités cosinus entre la requête et les documents
    cosine_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    # Obtenir les top n résultats
    top_results = torch.topk(cosine_scores, k=top_n)
    result_indices = top_results.indices.tolist()
    scores = top_results.values.tolist()
    return corpus_df.iloc[result_indices].assign(semantic_score=scores)

query = "deep learning"
top_results_semantic = semantic_search(query, model, corpus_embeddings, pdf_data_df)
print(top_results_semantic[['title', 'semantic_score']])


                                                title  semantic_score
31                                                NaN        0.270222
36  s des programmes. Afin d’´eviter au lecteur un...        0.270222
48  de DOCTEUR EN INFORMATIQUE par Caroline LE CAL...        0.270222
18  . III.Series. QA188.B462003 512.9′434—dc21 200...        0.270222
45  quiresafixedamountofcompu- tational work at ea...        0.270222
